## Imports

In [2]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta

## Data Read-in

In [3]:
df = pd.read_csv('2023 eviction notices.xlsx - Sheet1.csv')

In [6]:
df['File Date'] = pd.to_datetime(df['File Date'])

In [9]:
df_filtered = df[df['File Date'] > '5.31.23']

In [11]:
df_filtered.columns

Index(['Eviction ID', 'Address', 'City', 'State',
       'Eviction Notice Source Zipcode', 'File Date', 'Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends', 'Constraints Date', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Location', 'Shape',
       'SF Find Neighborhoods', 'Current Police Districts',
       'Current Supervisor Districts', 'Analysis Neighborhoods',
       'DELETE - Neighborhoods', 'DELETE - Police Districts',
       'DELETE - Supervisor Districts', 'DELETE - Fire Prevention Districts',
       'DELETE - Zip Codes', 'CBD, BID and GBD Boundaries as of 2017',
       'Central Market/Tenderloin Boundary', 'Areas of Vu

In [13]:
columns_to_check = ['Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends']

In [14]:
df_filtered['Eviction_Reason'] = df_filtered[columns_to_check].idxmax(axis=1)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_73573/1230607948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Eviction_Reason'] = df_filtered[columns_to_check].idxmax(axis=1)


In [19]:
df_filtered['Eviction_Reason'].value_counts()

Eviction_Reason
Nuisance                641
Capital Improvement     190
Non Payment             175
Owner Move In           138
Breach                  129
Ellis Act WithDrawal     76
Other Cause              46
Roommate Same Unit       31
Unapproved Subtenant     27
Access Denial             6
Late Payments             6
Illegal Use               1
Name: count, dtype: int64

## HTML Popup Formatter

In [ ]:
df

In [46]:
def popup_html(row):
    Eviction_ID = row['Eviction ID']
    Address = row['Address']
    Neighborhood = row['Neighborhoods - Analysis Boundaries']
    Eviction_Reason = row['Eviction_Reason']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Eviction ID: </strong>{}'''.format(Eviction_ID) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Eviction Reason: </strong>{}'''.format(Eviction_Reason) + '''<br>
    </html>
    '''
    return html

In [47]:
df_filtered.columns

Index(['Eviction ID', 'Address', 'City', 'State',
       'Eviction Notice Source Zipcode', 'File Date', 'Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends', 'Constraints Date', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Location', 'Shape',
       'SF Find Neighborhoods', 'Current Police Districts',
       'Current Supervisor Districts', 'Analysis Neighborhoods',
       'DELETE - Neighborhoods', 'DELETE - Police Districts',
       'DELETE - Supervisor Districts', 'DELETE - Fire Prevention Districts',
       'DELETE - Zip Codes', 'CBD, BID and GBD Boundaries as of 2017',
       'Central Market/Tenderloin Boundary', 'Areas of Vu

In [48]:
# Remove parentheses and split the 'Location' column
df_filtered[['LATITUDE', 'LONGITUDE']] = df_filtered['Location'].str.replace('[()]', '', regex=True).str.split(', ', expand=True)

# Convert 'LATITUDE' and 'LONGITUDE' to numeric types
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'], errors='coerce')
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'], errors='coerce')

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_73573/4155439623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[['LATITUDE', 'LONGITUDE']] = df_filtered['Location'].str.replace('[()]', '', regex=True).str.split(', ', expand=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_73573/4155439623.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'], errors='coerce')
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000

In [49]:
len(df_filtered)

1182

In [50]:
df_filtered = df_filtered.dropna(subset=['LATITUDE','LONGITUDE'])

In [51]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=11,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"San Francisco Eviction Notices - July 1st to Dec. 31st")

m.get_root().html.add_child(folium.Element(title_html))

for index, row in df_filtered.iterrows():
    marker = folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

# Add the FeatureGroups to the map

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [70]:
m.save('index.html')

## Summary Info

In [71]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [72]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [73]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [74]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Hillsboro-Beach/1187-Hillsboro-Mile-33062/unit-4E/home/41515941


In [75]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Las Olas River House, 333 Las Olas Way Ph 3210 Fort Lauderdale | Price $7,200,000 | $1,042 psf | Year built: 2004
Least Expensive
Sunrisw Lakes Condo, 2700 W Sunrise Lakes Dr #105 Sunrise | Price $47,509 | $74 psf | Year built: 1972
Highest Price Per Square Foot
Four Seasons, 525 N Ft Lauderdale Bch Blvd #1608 Fort Lauderdale | Price $3,200,000 | $1,723 psf | Year built: 2019
Lowest Price Per Square Foot
Sunrisw Lakes Condo, 2700 W Sunrise Lakes Dr #105 Sunrise | Price $47,509 | $74 psf | Year built: 1972
Newest
Solemar Beach Condos, 1116 N Ocean Blvd #1204 Pompano Beach | Price $2,595,000 | $1,133 psf | Year built: 2023
Oldest
Virginia Kaye East, 1187 Hillsboro Mile Unit 4E Hillsboro Beach | Price $850,000 | $567 psf | Year built: 1950


## Map URL Snagger

In [76]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/BC_condo_sales_month_ending_dec_2023


## Get Summary Data

In [77]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 738
--------
Total sale price: $255,630,664
--------
Median sale price: $240,000
--------
Max sale price: $7,200,000
--------
Min sale price: $47,509
------------------------------------------------
PSF INFO
Max price per square foot: $1,723
--------
Min price per square foot: $74
--------
Median price per square foot: $236
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1950.0
----------
Average building age: 1980.9322493224931


In [78]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

# GENERATE STORY TEMPLATE

# STOP! DID YOU UPDATE VALUES FROM THE PREVIOUS MONTH?

In [103]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = sales_volume_2nd_month  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = total_condos_sold_2nd_month

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    social = 'higher'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    social = 'lower'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    social = 'equal'
    
    
if subject_month_closings > prior_month_closings:
    dek_rf = 'rises'
elif prior_month_closings > subject_month_closings:
    dek_rf ='falls'
else:
    dek_rf = 'equals'

In [106]:
story_string = f'''
\033[1mHED:\033[0m Broward County condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Broward County {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m Broward County’s condo sales volume {hed_rf} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m Broward County had ${round(df_filtered["PRICE"].sum()/1_000_000)} million in condo sales in {previous_month_name}, {social} than ${round(sales_volume_2nd_month/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Broward County, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m


Broward County’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${sales_volume_2nd_month/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, down from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max()/1_000_000:.0f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${median_sales_price_2nd_month:,}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max()/1_000_000:.1f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. WHERE XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [107]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")


HED: Broward County condo sales dollar volume drops in December to $256M 
DEK: Number of closings falls to 738 from 829 in November
FEATURED HED:
SEO HED: Broward County December Condo Sales Report 
SEO DESCRIPTION: Broward County’s condo sales volume drops to $256 million in December.
AUTHOR: Adam Farence
RESEARCH: 
Social: Broward County had $256 million in condo sales in December, lower than $286 million from November, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, monthly condo sales, condos, condo market, Las Olas River House, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:


Broward County’s NEWS PEG HERE.

December condo sales totaled $256 million, falling from $286 million in November. Brokers closed 738 sales last month, down from 829 sales in November and XXX in XXX, Multiple Listing

# Print links for notable sales

## Top sale

In [82]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/333-Las-Olas-Way-33301/unit-3210/home/42028562


## Cheapest sale

In [83]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Sunrise/2700-W-Sunrise-Lakes-Dr-33322/unit-105/home/41607460


## Highest PSF

In [84]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-1608/home/145299255


## Lowest PSF

In [85]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Sunrise/2700-W-Sunrise-Lakes-Dr-33322/unit-105/home/41607460


## Newest

In [86]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Pompano-Beach/1116-N-Ocean-Blvd-33062/unit-1204/home/187605253


## Oldest

In [87]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Hillsboro-Beach/1187-Hillsboro-Mile-33062/unit-4E/home/41515941


## Time on Market Calculator

In [101]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 10, 27) ## List (Earlier) date
date2 = datetime(2023, 12, 8) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

42
